In [1]:
%%capture
!pip install pip --upgrade
!pip install gigachain-cli
!gigachain install-rus-certs
!pip install sentence-transformers
!pip install faiss-cpu
!pip install rank_bm25
!pip install datasets -q

!pip install pytelegrambotapi
!pip install gigachain
!pip install gigachat
!pip install gigachain-cli
!pip install notifiers
!pip install telebot

In [1]:
CREDS = 'NWExYjc1NzAtMzYwMi00ODdlLWJmMGMtZGU0NWU5YjUwZDk4OjVlNDE0OTkwLTBkOTItNDIzOC1hM2JhLWI1MDhkYjE5NDZiNw=='
TG_API_TOKEN = '6414200086:AAHAXaaSbGYhx1IfMpo7s4SBRSdUxLqaGyE'

In [2]:
from datasets import load_dataset, Dataset, DatasetDict
import os
import json
import pathlib
import string
from typing import Any, Coroutine, List
from langchain.docstore.base import Document
from langchain.chains import RetrievalQA
from langchain.schema import HumanMessage, SystemMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
import telebot
from telebot import types
from google.colab import userdata

!!!УКАЗАТЬ ПУТЬ ДО dataset_max.json!!!

In [3]:
path_dataset = pathlib.Path('dataset_max.json')

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

In [5]:
with open(path_dataset) as f:
  my_dataset = json.load(f)

In [6]:
map_datasets_all = {}

for key in my_dataset:
  map_datasets_all[key] = set()
  index_in = 0
  index_out = 900

  while True:
    if len(my_dataset[key]) > index_out:
      map_datasets_all[key].add(my_dataset[key][index_in:index_out])
    else:
      map_datasets_all[key].add(my_dataset[key][index_in:])
      break

    index_in += 700
    index_out += 700

In [7]:
new_dataset = set()
for key in map_datasets_all:
  for one_set in map_datasets_all[key]:
    new_dataset.add(one_set)

In [8]:
len(new_dataset)

3261

In [9]:
documents = [
    Document(page_content=context) for context in new_dataset
]

In [10]:
def get_name_dataset(text_doc) -> str:
  for key in map_datasets_all:
    if text_doc in map_datasets_all[key]:
      return key

  return None

In [11]:
class HuggingFaceE5Embeddings(HuggingFaceEmbeddings):
    def embed_query(self, text: str) -> List[float]:
        text = f"query: {text}"
        return super().embed_query(text)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        texts = [f"passage: {text}" for text in texts]
        return super().embed_documents(texts)

    async def aembed_query(self, text: str) -> Coroutine[Any, Any, List[float]]:
        text = f"query: {text}"
        return await super().aembed_query(text)

    async def aembed_documents(
        self, texts: List[str]
    ) -> Coroutine[Any, Any, List[List[float]]]:
        texts = [f"passage: {text}" for text in texts]
        return await super().aembed_documents(texts)

In [12]:
embedding = HuggingFaceE5Embeddings(model_name="intfloat/multilingual-e5-base")

In [13]:
faiss_db = FAISS.from_documents(documents, embedding=embedding)

In [14]:
embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 5})

In [15]:
def acc_top(dataset: Dataset, right_column: str, answer_column: str) -> float:
    temp_dataset = dataset.map(
        lambda x: {"is_right_retrieved": x[right_column] in x[answer_column]}
    )
    return sum(temp_dataset["is_right_retrieved"]) / len(temp_dataset)

In [16]:
def tokenize(s: str) -> list[str]:
    """Очень простая функция разбития предложения на слова"""
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")

embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 2})
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=3,
)
ensemble_retriever = EnsembleRetriever(
    retrievers=[embedding_retriever, bm25_retriever],
    weights=[0.4, 0.6],
)

In [17]:
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials=CREDS, verify_ssl_certs=False)

bot = telebot.TeleBot(TG_API_TOKEN)

keyboard = types.InlineKeyboardMarkup()
keyboard.add(
    types.InlineKeyboardButton(text='👍', callback_data='button_pressed_Ok'),
    types.InlineKeyboardButton(text='👎', callback_data='button_pressed_error')
)

instrucrion = 'Отвечай кратко. Желательно в одном предложении'

In [18]:
@bot.callback_query_handler(func=lambda call: True)
def callback_handler(call):
    if call.data == 'button_pressed_Ok':
        bot.answer_callback_query(call.id, 'Ok! Я запомню🌟')
    elif call.data == 'button_pressed_error':
      temp_message_id = bot.send_message(call.message.json['chat']['id'], 'Ok! Попробую еще раз. Ожидайте...').message_id
      bot.answer_callback_query(call.id, '😫😫😫')

      if 'reply_to_message' in call.message.json:
        text = call.message.json['reply_to_message']['text']
      else:
        text = call.message.json['text']
      messages = [
          SystemMessage(content=instrucrion),
          HumanMessage(content=text)
      ]
      res = chat(messages)
      messages.append(res)
      bot.send_message(call.message.json['chat']['id'], res.content)
      bot.delete_message(call.message.json['chat']['id'], temp_message_id)

In [19]:
@bot.message_handler(func=lambda message: True)
def echo_message(message):
  temp_message_id = bot.send_message(message.chat.id, 'Ожидайте, запрос выполняется...').message_id
  giga_answer = qa.invoke({"query": message.text})

  answer_text = f'''{giga_answer['result']}\r\nОтвет найден в "{get_name_dataset(giga_answer['source_documents'][0].page_content)}"'''
  bot.reply_to(message, answer_text, reply_markup=keyboard)
  bot.delete_message(message.chat.id, temp_message_id)

In [20]:
from langchain.llms.gigachat import GigaChat

llm = GigaChat(credentials=CREDS)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True,
)

In [22]:
bot.polling()